In [12]:
# creating paths to src and data folders in the repo
import sys
import pathlib
src_path = pathlib.Path().absolute().parent.parent / "src"
data_path = pathlib.Path().absolute().parent.parent / "data"

# add src path to sys.path so it is searched in import statements
sys.path.append(str(src_path))

# basic imports for data retreival and manipulation
import requests
import pandas as pd
import numpy as np
import datetime as dt
import time

# importing api wrappers for reddit data
import praw
from psaw import PushshiftAPI

# retrieve api credentials from .gitignore'd text file
secrets_path = src_path / 'api_credentials.txt'
secrets_txt = open(secrets_path, 'r')

my_id = secrets_txt.readline().split('=')[1].rstrip()
my_secret = secrets_txt.readline().split('=')[1].rstrip()

secrets_txt.close()

# create a praw and pushshitft instances
reddit = praw.Reddit(
     client_id=my_id,
     client_secret=my_secret,
     user_agent="test_script by u/Mizule_RL"
 )

s_api = PushshiftAPI(reddit)


In [15]:
import re

I'm choosing to limit my classes to: Digital, Paint, Ink, Pencil, Charcoal, Sculpture

In [22]:
def get_wordlists():
    '''
    returns a dictionary containing the identifying words for each class
    '''
    wordlists = {'digital': ['digital', 'adobe', 'photoshop', 'procreate', 'wacom', 'tablet', 'illustrator', '3d', 'vector'],
                 'paint': ['acrylic', 'oil', 'watercolor', 'water color'],
                 'ink': ['pen', 'marker', 'ink'],
                 'pencil': ['pencil', 'colored pencil', 'coloredpencil', 'graphite'],
                 'charcoal': ['charcoal'],
                 'sculpture': ['clay', 'string', 'sculpture', 'wire', 'nail', 'glass', 'yarn', 'metal', 'copper']}
    return wordlists

In [32]:
def validate_submission(post):
    '''
    function for determining if a post is formated in a way that can be parsed properly
    
    Parameters:
    -- post: praw.submission object to be validated
    
    Returns:
    -- Boolean: True if post has valid formatting, False if not
    '''
    
    link_format = 'https://i\.redd\.it/.{13}\.(jpg|png)'
    corpus = []
    wordlists = get_wordlists()
    for value in wordlists.values():
        corpus.extend(value)
    title = post.title.lower().split(',')
    if len(title) == 4:
        medium = title[2].strip()
        if medium == 'me':
            medium = title[1].strip()
        if re.match(link_format, post.url):
            for word in corpus:
                if medium.find(word) >= 0:
                    return True
    return False
    
def extract_medium_from_title(title):
    '''
    identifies the medium given the title of the post
    '''
    medium_counter = 0
    medium_type = 'other'
    title_split = title.lower().split(',')
    medium = title_split[2].strip()
    if medium == 'me':
        medium = title_split[1].strip()
    wordlists = get_wordlists()
    for item in wordlists.items():
        for word in item[1]:
            if medium.find(word) >= 0:
                medium_type = item[0]
                medium_counter += 1
                break
    if medium_counter > 1:
        return 'mixed_medium'
    else:
        return medium_type

In [38]:
def make_post_dict(post):
    '''
    creates and returns a dict containing relevant data from a post
    '''
    post_dict = {'title': post.title,
                 'medium': extract_medium_from_title(post.title),
                 'url': post.url, 
                 'id': post.id,
                 'unix_time': int(post.created_utc)}
    return post_dict

In [29]:
def fetch_submissions(min_posts, date = int(dt.datetime.now().timestamp())):
    start_epoch = date
    data_size = min_posts

    collected_posts = []
    while len(collected_posts) < data_size:
        print(f'polling pushshift for {data_size - len(collected_posts)} more posts before {start_epoch}')
        batch = list(s_api.search_submissions(before=start_epoch, subreddit='Art', limit=1000))
        for post in batch:
            if validate_submission(post):
                collected_posts.append(post)
        start_epoch = int(batch[-1].created_utc)
    return collected_posts

In [30]:
submissions_raw = fetch_submissions(10000)

polling pushshift for 10000 more posts before 1605647143
polling pushshift for 9959 more posts before 1605599241
polling pushshift for 9917 more posts before 1605585967
polling pushshift for 9862 more posts before 1605574629
polling pushshift for 9822 more posts before 1605565187
polling pushshift for 9783 more posts before 1605559306
polling pushshift for 9739 more posts before 1605551360
polling pushshift for 9694 more posts before 1605543522
polling pushshift for 9651 more posts before 1605537366
polling pushshift for 9602 more posts before 1605527150
polling pushshift for 9556 more posts before 1605512188
polling pushshift for 9512 more posts before 1605499991
polling pushshift for 9468 more posts before 1605488196
polling pushshift for 9413 more posts before 1605478748
polling pushshift for 9357 more posts before 1605470480
polling pushshift for 9305 more posts before 1605463098
polling pushshift for 9256 more posts before 1605455457
polling pushshift for 9197 more posts before 16

polling pushshift for 3074 more posts before 1603929914
polling pushshift for 3017 more posts before 1603922634
polling pushshift for 2971 more posts before 1603917654
polling pushshift for 2917 more posts before 1603910415
polling pushshift for 2868 more posts before 1603902956
polling pushshift for 2827 more posts before 1603895308
polling pushshift for 2779 more posts before 1603881442
polling pushshift for 2733 more posts before 1603859298
polling pushshift for 2687 more posts before 1603847110
polling pushshift for 2638 more posts before 1603837388
polling pushshift for 2594 more posts before 1603828840
polling pushshift for 2548 more posts before 1603822944
polling pushshift for 2499 more posts before 1603813353
polling pushshift for 2448 more posts before 1603804778
polling pushshift for 2394 more posts before 1603791495
polling pushshift for 2353 more posts before 1603774306
polling pushshift for 2310 more posts before 1603763542
polling pushshift for 2266 more posts before 160

In [39]:
submissions_dict = [make_post_dict(post) for post in submissions_raw]

In [40]:
submissions_dict

[{'title': 'Diya, me, 3D, 2020',
  'medium': 'digital',
  'url': 'https://i.redd.it/dk2s8oj6hsz51.jpg',
  'id': 'jvrq4w',
  'unix_time': 1605614029},
 {'title': 'Hexes&Exes, Webtoon, Digital Art, 2020',
  'medium': 'digital',
  'url': 'https://i.redd.it/o3m1hkvnesz51.png',
  'id': 'jvrk3v',
  'unix_time': 1605613143},
 {'title': 'Onward, me, ink on bristol, 2020',
  'medium': 'ink',
  'url': 'https://i.redd.it/0c4926h8dsz51.jpg',
  'id': 'jvrgof',
  'unix_time': 1605612655},
 {'title': 'Explosion, Me, Digital, 2020',
  'medium': 'digital',
  'url': 'https://i.redd.it/3txx4lcqbsz51.jpg',
  'id': 'jvrd6y',
  'unix_time': 1605612143},
 {'title': 'Lily, Me, Pen, 2020',
  'medium': 'ink',
  'url': 'https://i.redd.it/jxhc3pnmbsz51.jpg',
  'id': 'jvrczf',
  'unix_time': 1605612111},
 {'title': 'Imposter syndrome, me, digital, 2020',
  'medium': 'digital',
  'url': 'https://i.redd.it/buyuzygebsz51.jpg',
  'id': 'jvrcgp',
  'unix_time': 1605612035},
 {'title': 'White Rhino Track, Me, acrylic in

In [41]:
submissions_df = pd.DataFrame(submissions_dict)

In [42]:
submissions_df

,title,medium,url,id,unix_time
0,"Diya, me, 3D, 2020",digital,https://i.redd.it/dk2s8oj6hsz51.jpg,jvrq4w,1605614029
1,"Hexes&Exes, Webtoon, Digital Art, 2020",digital,https://i.redd.it/o3m1hkvnesz51.png,jvrk3v,1605613143
2,"Onward, me, ink on bristol, 2020",ink,https://i.redd.it/0c4926h8dsz51.jpg,jvrgof,1605612655
3,"Explosion, Me, Digital, 2020",digital,https://i.redd.it/3txx4lcqbsz51.jpg,jvrd6y,1605612143
4,"Lily, Me, Pen, 2020",ink,https://i.redd.it/jxhc3pnmbsz51.jpg,jvrczf,1605612111
...,...,...,...,...,...
10031,"Untitled, me, acrylic on canvas board, 2020",paint,https://i.redd.it/hkzkihqtyau51.jpg,jewxsy,1603222750
10032,"Wire wrapped crystal pendant, me, argentium si...",sculpture,https://i.redd.it/fp8aqleqyau51.jpg,jewxdr,1603222716
10033,"“Lost”, Artist me, Digital Painting, 2020",digital,https://i.redd.it/5zrygut4yau51.jpg,jewv1b,1603222527
10034,"The great egret, me, watercolor, 2020",paint,https://i.redd.it/3pw64tibxau51.jpg,jewro8,1603222239


In [43]:
submissions_df.medium.value_counts()

digital         5094
paint           2484
mixed_medium    1145
ink              949
pencil           158
charcoal         104
sculpture        102
Name: medium, dtype: int64

In [48]:
submissions_df[submissions_df.id == 'jjlfqf']

,title,medium,url,id,unix_time
7219,"Gluttony, Nazeroo(me), Digital Art, 2020",digital,https://i.redd.it/o5c323tydtv51.jpg,jjlfqf,1603881778


In [50]:
submissions_dict[7220]

{'title': 'Self-portrait, Me, Digital, 2020',
 'medium': 'digital',
 'url': 'https://i.redd.it/jrhr0vpbdtv51.png',
 'id': 'jjldco',
 'unix_time': 1603881445}

In [51]:
remaining_posts = submissions_dict[7220:]

In [52]:
count = 0
for post in remaining_posts:
    response = requests.get(post['url'])
    file_name = f"{post['id']}.{post['url'][-3:]}"
    file_path = data_path / 'initial_dump' / post['medium'] / file_name
    print(f'writting file: {file_name} | {len(remaining_posts) - count} images remaining')
    file = open(str(file_path), "wb")
    file.write(response.content)
    file.close()
    count += 1

writting file: jjldco.png | 2816 images remaining
writting file: jjkx8c.jpg | 2815 images remaining
writting file: jjkwkv.jpg | 2814 images remaining
writting file: jjkw2i.jpg | 2813 images remaining
writting file: jjks9m.jpg | 2812 images remaining
writting file: jjkqe4.jpg | 2811 images remaining
writting file: jjkoza.jpg | 2810 images remaining
writting file: jjklkw.jpg | 2809 images remaining
writting file: jjkiwi.jpg | 2808 images remaining
writting file: jjkir3.jpg | 2807 images remaining
writting file: jjkgao.jpg | 2806 images remaining
writting file: jjkews.jpg | 2805 images remaining
writting file: jjkbx5.jpg | 2804 images remaining
writting file: jjk6xj.jpg | 2803 images remaining
writting file: jjk4yl.jpg | 2802 images remaining
writting file: jjk48n.jpg | 2801 images remaining
writting file: jjk1np.jpg | 2800 images remaining
writting file: jjjy8r.jpg | 2799 images remaining
writting file: jjjqc2.jpg | 2798 images remaining
writting file: jjjo1j.jpg | 2797 images remaining


writting file: jj9rdn.jpg | 2652 images remaining
writting file: jj9otn.jpg | 2651 images remaining
writting file: jj9n7j.png | 2650 images remaining
writting file: jj9luh.jpg | 2649 images remaining
writting file: jj9js0.jpg | 2648 images remaining
writting file: jj9eet.png | 2647 images remaining
writting file: jj9cka.png | 2646 images remaining
writting file: jj9c8l.jpg | 2645 images remaining
writting file: jj99mf.jpg | 2644 images remaining
writting file: jj9889.png | 2643 images remaining
writting file: jj97uv.jpg | 2642 images remaining
writting file: jj97ak.jpg | 2641 images remaining
writting file: jj94xw.jpg | 2640 images remaining
writting file: jj91vo.jpg | 2639 images remaining
writting file: jj90oe.png | 2638 images remaining
writting file: jj90ms.jpg | 2637 images remaining
writting file: jj8xl7.jpg | 2636 images remaining
writting file: jj8vaa.jpg | 2635 images remaining
writting file: jj8rx0.jpg | 2634 images remaining
writting file: jj8o6j.jpg | 2633 images remaining


writting file: jj159e.jpg | 2487 images remaining
writting file: jj13b9.jpg | 2486 images remaining
writting file: jj0ypf.png | 2485 images remaining
writting file: jj0yon.jpg | 2484 images remaining
writting file: jj0s2o.png | 2483 images remaining
writting file: jj0rco.png | 2482 images remaining
writting file: jj0qr1.jpg | 2481 images remaining
writting file: jj0qjx.png | 2480 images remaining
writting file: jj0ms0.jpg | 2479 images remaining
writting file: jj0k8r.jpg | 2478 images remaining
writting file: jj0gdl.png | 2477 images remaining
writting file: jj0bas.jpg | 2476 images remaining
writting file: jj09fo.jpg | 2475 images remaining
writting file: jj093z.jpg | 2474 images remaining
writting file: jj05cc.jpg | 2473 images remaining
writting file: jj03zd.jpg | 2472 images remaining
writting file: jj02to.jpg | 2471 images remaining
writting file: jj01h2.jpg | 2470 images remaining
writting file: jizzke.jpg | 2469 images remaining
writting file: jizz93.jpg | 2468 images remaining


writting file: jir7vt.jpg | 2323 images remaining
writting file: jir77k.jpg | 2322 images remaining
writting file: jir4p0.jpg | 2321 images remaining
writting file: jir2nr.jpg | 2320 images remaining
writting file: jir10g.jpg | 2319 images remaining
writting file: jir0qq.jpg | 2318 images remaining
writting file: jiqy63.jpg | 2317 images remaining
writting file: jiqxq1.jpg | 2316 images remaining
writting file: jiqmck.jpg | 2315 images remaining
writting file: jiql1w.jpg | 2314 images remaining
writting file: jiqkli.png | 2313 images remaining
writting file: jiqhzh.jpg | 2312 images remaining
writting file: jiqeb6.jpg | 2311 images remaining
writting file: jiqbre.jpg | 2310 images remaining
writting file: jiq99z.jpg | 2309 images remaining
writting file: jiq8t3.jpg | 2308 images remaining
writting file: jiq8cf.jpg | 2307 images remaining
writting file: jiq78x.jpg | 2306 images remaining
writting file: jiq3jh.jpg | 2305 images remaining
writting file: jiq2ui.jpg | 2304 images remaining


writting file: jigbyt.jpg | 2159 images remaining
writting file: jig1fo.png | 2158 images remaining
writting file: jifxgh.jpg | 2157 images remaining
writting file: jifw9y.jpg | 2156 images remaining
writting file: jifvm9.png | 2155 images remaining
writting file: jifs91.jpg | 2154 images remaining
writting file: jifry2.jpg | 2153 images remaining
writting file: jifq6k.jpg | 2152 images remaining
writting file: jifokz.jpg | 2151 images remaining
writting file: jifoh5.png | 2150 images remaining
writting file: jifo05.jpg | 2149 images remaining
writting file: jifnza.jpg | 2148 images remaining
writting file: jifk5t.png | 2147 images remaining
writting file: jifi37.png | 2146 images remaining
writting file: jiffk3.png | 2145 images remaining
writting file: jife9d.jpg | 2144 images remaining
writting file: jifcif.jpg | 2143 images remaining
writting file: jifaw7.jpg | 2142 images remaining
writting file: jifaim.jpg | 2141 images remaining
writting file: jif9ty.jpg | 2140 images remaining


...

writting file: jg94ni.jpg | 840 images remaining
writting file: jg92wr.jpg | 839 images remaining
writting file: jg90w5.jpg | 838 images remaining
writting file: jg90to.jpg | 837 images remaining
writting file: jg90pz.jpg | 836 images remaining
writting file: jg8zme.jpg | 835 images remaining
writting file: jg8s3l.jpg | 834 images remaining
writting file: jg8nxm.png | 833 images remaining
writting file: jg8hp0.jpg | 832 images remaining
writting file: jg8f8v.jpg | 831 images remaining
writting file: jg80zc.jpg | 830 images remaining
writting file: jg7x96.jpg | 829 images remaining
writting file: jg7wfb.jpg | 828 images remaining
writting file: jg7vpd.jpg | 827 images remaining
writting file: jg7v3i.jpg | 826 images remaining
writting file: jg7u5g.jpg | 825 images remaining
writting file: jg7oc5.jpg | 824 images remaining
writting file: jg7njn.png | 823 images remaining
writting file: jg7n9e.png | 822 images remaining
writting file: jg7mdi.jpg | 821 images remaining
writting file: jg7kz

writting file: jfzhzv.jpg | 672 images remaining
writting file: jfzgi3.png | 671 images remaining
writting file: jfze23.jpg | 670 images remaining
writting file: jfzcpq.jpg | 669 images remaining
writting file: jfz60y.jpg | 668 images remaining
writting file: jfz5v7.png | 667 images remaining
writting file: jfz4te.jpg | 666 images remaining
writting file: jfyylr.jpg | 665 images remaining
writting file: jfyt9y.jpg | 664 images remaining
writting file: jfyqvs.jpg | 663 images remaining
writting file: jfypo5.png | 662 images remaining
writting file: jfykxd.png | 661 images remaining
writting file: jfyiyh.jpg | 660 images remaining
writting file: jfyi7z.png | 659 images remaining
writting file: jfyh6p.jpg | 658 images remaining
writting file: jfydl2.jpg | 657 images remaining
writting file: jfyc2z.jpg | 656 images remaining
writting file: jfybxz.jpg | 655 images remaining
writting file: jfyayp.jpg | 654 images remaining
writting file: jfya73.png | 653 images remaining
writting file: jfy4e

writting file: jfp6jx.png | 504 images remaining
writting file: jfp4wj.jpg | 503 images remaining
writting file: jfp1tv.jpg | 502 images remaining
writting file: jfp16q.jpg | 501 images remaining
writting file: jfozpg.jpg | 500 images remaining
writting file: jfoxh7.jpg | 499 images remaining
writting file: jfowub.jpg | 498 images remaining
writting file: jfovcs.jpg | 497 images remaining
writting file: jfoneb.jpg | 496 images remaining
writting file: jfokxt.jpg | 495 images remaining
writting file: jfoih9.png | 494 images remaining
writting file: jfofv2.png | 493 images remaining
writting file: jfocu9.jpg | 492 images remaining
writting file: jfoc0r.jpg | 491 images remaining
writting file: jfo6tk.jpg | 490 images remaining
writting file: jfo5cm.jpg | 489 images remaining
writting file: jfo0nc.jpg | 488 images remaining
writting file: jfo054.jpg | 487 images remaining
writting file: jfnxgl.png | 486 images remaining
writting file: jfnwne.jpg | 485 images remaining
writting file: jfnsw

writting file: jfgb0r.jpg | 335 images remaining
writting file: jfg8ab.jpg | 334 images remaining
writting file: jfg4io.png | 333 images remaining
writting file: jfg2wl.png | 332 images remaining
writting file: jffzd2.jpg | 331 images remaining
writting file: jffxst.jpg | 330 images remaining
writting file: jffvyq.jpg | 329 images remaining
writting file: jffsso.png | 328 images remaining
writting file: jffsf6.png | 327 images remaining
writting file: jffs3n.png | 326 images remaining
writting file: jffqlg.png | 325 images remaining
writting file: jffqhf.jpg | 324 images remaining
writting file: jffplm.png | 323 images remaining
writting file: jffnxd.jpg | 322 images remaining
writting file: jffljf.jpg | 321 images remaining
writting file: jffl7f.jpg | 320 images remaining
writting file: jffiwk.png | 319 images remaining
writting file: jffg5p.png | 318 images remaining
writting file: jffg4d.png | 317 images remaining
writting file: jffer5.jpg | 316 images remaining
writting file: jffe2

writting file: jf79nd.jpg | 167 images remaining
writting file: jf78ia.png | 166 images remaining
writting file: jf77ut.png | 165 images remaining
writting file: jf76pi.png | 164 images remaining
writting file: jf75uz.png | 163 images remaining
writting file: jf75om.jpg | 162 images remaining
writting file: jf718z.jpg | 161 images remaining
writting file: jf6qet.png | 160 images remaining
writting file: jf6nz6.jpg | 159 images remaining
writting file: jf6lpz.jpg | 158 images remaining
writting file: jf6h4d.jpg | 157 images remaining
writting file: jf6gmp.jpg | 156 images remaining
writting file: jf6gkk.jpg | 155 images remaining
writting file: jf667o.jpg | 154 images remaining
writting file: jf61vw.jpg | 153 images remaining
writting file: jf60f4.jpg | 152 images remaining
writting file: jf5ymu.jpg | 151 images remaining
writting file: jf5xhq.jpg | 150 images remaining
writting file: jf5vgm.jpg | 149 images remaining
writting file: jf5qf6.jpg | 148 images remaining
writting file: jf5q9